# Hyperparameter Optimization

This is the last notebook of this project. Here I'll try to squeeze a little more AUC from LightGBM by tweaking its hyperparameters. I'll use Bayesian Search here, as it usually performs better than Random or Grid Search.

In [18]:
from functools import partial
from sklearn import ensemble, metrics, model_selection
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
sns.set()
import warnings
warnings.filterwarnings(action='ignore')
import gc
from imports import *

In [3]:
df=pd.read_csv('../data/train_fs.csv')
df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_WORK_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_CITY_NOT_LIVE_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,WALLSMATERIAL_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,NEW_AGE_GROUP,NEW_AMT_CREDIT_TO_AMT_INCOME,NEW_AMT_CREDIT_TO_AMT_ANNUITY,NEW_AMT_CREDIT_TO_AMT_GOODS_PRICE,NEW_DOC_FLAG_MEAN,NEW_DAYS_REGISTRATION_TO_DAYS_BIRTH,NEW_EXT_SOURCES_MEAN,NEW_EXT_SOURCES_STD,bureau_DAYS_CREDIT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_CREDIT_SUM_DEBT_min,bureau_AMT_CREDIT_MAX_OVERDUE_count,bureau_AMT_CREDIT_SUM_LIMIT_mean,bureau_AMT_CREDIT_SUM_LIMIT_sum,bureau_DAYS_ENDDATE_FACT_count,bureau_AMT_CREDIT_SUM_LIMIT_count,bureau_DAYS_CREDIT_ENDDATE_mean,bureau_DAYS_CREDIT_ENDDATE_sum,bureau_DAYS_CREDIT_ENDDATE_max,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Carloan_count_norm,bureau_CREDIT_TYPE_Creditcard_count_norm,bureau_CREDIT_TYPE_Creditcard_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_TYPE_Consumercredit_count_norm,bureau_CREDIT_ACTIVE_Active_count,loan_bureau_balance_MONTHS_BALANCE_sum_sum,loan_bureau_balance_MONTHS_BALANCE_sum_min,loan_bureau_balance_MONTHS_BALANCE_mean_min,loan_bureau_balance_MONTHS_BALANCE_sum_max,loan_bureau_balance_MONTHS_BALANCE_max_sum,loan_bureau_balance_MONTHS_BALANCE_max_min,loan_bureau_balance_MONTHS_BALANCE_max_mean,loan_bureau_balance_STATUS_1_count_norm_min,loan_bureau_balance_STATUS_X_count_norm_min,loan_bureau_balance_STATUS_X_count_min,loan_bureau_balance_STATUS_C_count_min,loan_bureau_balance_STATUS_1_count_norm_mean,loan_bureau_balance_STATUS_0_count_norm_min,loan_bureau_balance_STATUS_1_count_norm_max,loan_bureau_balance_STATUS_1_count_norm_sum,loan_bureau_balance_STATUS_1_count_mean,loan_bureau_balance_STATUS_X_count_norm_mean,loan_bureau_balance_STATUS_0_count_norm_mean,loan_bureau_balance_STATUS_C_count_norm_mean,loan_bureau_balance_STATUS_X_count_norm_max,loan_bureau_balance_STATUS_C_count_norm_max,loan_bureau_balance_STATUS_0_count_norm_max,loan_bureau_balance_STATUS_0_count_min,loan_bureau_balance_STATUS_1_count_max,loan_bureau_balance_STATUS_X_count_norm_sum,loan_bureau_balance_STATUS_0_count_norm_sum,loan_bureau_balance_STATUS_C_count_norm_sum,loan_bureau_balance_STATUS_X_count_mean,loan_bureau_balance_STATUS_0_count_mean,loan_bureau_balance_STATUS_X_count_max,loan_bureau_balance_STATUS_0_count_max,loan_bureau_balance_STATUS_C_count_mean,loan_bureau_balance_STATUS_X_count_sum,loan_bureau_balance_STATUS_0_count_sum,loan_bureau_balance_STATUS_C_count_max,previous_NAME_GOODS_CATEGORY_SportandLeisure_count_norm,previous_CODE_REJECT_REASON_SCOFR_count_norm,previous_NAME_CASH_LOAN_PURPOSE_Repairs_count_norm,previous_PRODUCT_COMBINATION_CashStreetmiddle_count_norm,previous_CHANNEL_TYPE_Channelofcorporatesales_count_norm,previous_CODE_REJECT_REASON_SCO_count_norm,previous_NAME_CONTRACT_STATUS_Unusedoffer_count_norm,previous_CODE_REJECT_REASON_HC_count_norm,previous_NAME_SELLER_INDUSTRY_Clothing_count_

In [29]:
def optimize(params, x, y, transformer, k=5):
    aucs_test=[]
    kfold=StratifiedKFold(k)
    for f, (tr, te) in enumerate(kfold.split(x, y=y)):
        X_train, y_train=x.iloc[tr, :], y.iloc[tr]
        X_test, y_test=x.iloc[te, :], y.iloc[te]

        X_train=transformer.fit_transform(X_train)
        X_test=transformer.transform(X_test)
        params=params
        dtrain=lgb.Dataset(X_train, label=y_train)
        dval=lgb.Dataset(X_test, label=y_test)

        params['metrics']='auc'
        for p in ['num_leaves', 'max_depth', 'min_child_samples']:
            params[p] = int(params[p])
            
        model=lgb.train(
                params=params,
                train_set=dtrain,
                valid_sets=[dtrain, dval],
                valid_names=['train', 'test'],
                callbacks=[lgb.early_stopping(100, verbose=-1), lgb.log_evaluation(-1)],
                verbose_eval=False)
        
        aucs_test.append(model.best_score['test']['auc'])

    return -1*np.mean(aucs_test)

In [31]:
df_sample=df.sample(frac=.2)

In [32]:
X, y=df_sample.drop(['TARGET'], axis=1), df_sample['TARGET']
cat_cols, num_cols=X.select_dtypes(include=['category', 'object']).columns, X.select_dtypes('number').columns
ohe=OneHotEncoder(sparse=False, handle_unknown='ignore')
col_tr=ColumnTransformer([
        ('cat', ohe, cat_cols),
        ('num', 'passthrough', num_cols)
    ])

param_space = {
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'max_depth': hp.quniform('max_depth', 3, 18, 1),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'is_unbalance': hp.choice('is_unbalance', [True, False]),
}

hopt=fmin(
    fn=partial(optimize, x=X, y=y, transformer=col_tr),
    space=param_space,
    algo=tpe.suggest,
    max_evals=150,
    trials=Trials()
)

print(hopt)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.133239 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56386                     
[LightGBM] [Info] Number of data points in the train set: 49201, number of used features: 419
[LightGBM] [Info] Start training from score 0.082620   
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit